# Multidimensional Polynomial Interpolation

In the {doc}`previous guide </getting-started/1d-polynomial-interpolation>`,
you learned the basics of polynomial interpolation in Minterpy for approximating one-dimensional functions.
As the name "Minterpy" suggests, the package also supports constructing multivariate (multidimensional) function.

In this in-depth tutorial, you will learn how to create an interpolating polynomial that approximates a two-dimensional function.
We use a two-dimensional example for ease of visualization, but the main steps you'll learn are applicable to polynomials of higher dimensions as well.

Before we begin, let's import the necessary packages to follow along with this guide.

In [ ]:
import minterpy as mp
import numpy as np
import matplotlib.pyplot as plt

## Motivating function

Consider the following two-dimensional function:

$$
\begin{align}
	\mathcal{M}(\boldsymbol{x}) = & 0.75 \exp{\left( -0.25 \left( (x_1 - 2)^2 + (x_2 - 2)^2 \right) \right) } \\
                                  & + 0.75 \exp{\left( -1.00 \left( \frac{(x_1 + 1)^2}{49} + \frac{(x_2 + 1)^2}{10} \right) \right)} \\
								  & + 0.50 \exp{\left( -0.25 \left( (x_1 - 7)^2 + (x_2 - 3)^2 \right) \right)} \\
								  & - 0.20 \exp{\left( -1.00 \left( (x_1 - 4)^2 + (x_2 - 7)^2 \right) \right)} \\
\end{align}
$$

where $\boldsymbol{x} = (x_1, x_2) \in [-1, 1]^2$.

```{note}
This function is known in the literature as the Franke function[^franke]. Here, however, the domain has been redefined to be $[-1, 1]^2$ instead of $[0, 1]^2$.

[^franke]: Richard Franke, "A critical comparison of some methods for interpolation of scattered data," Naval Postgraduate School, Monterey, Canada, Technical Report No. NPS53-79-003, 1979. URL: https://core.ac.uk/reader/36727660
```

The function can be defined in Python as follows:

In [ ]:
def fun(xx: np.ndarray):
    xx0 = 9 * xx[:, 0]
    xx1 = 9 * xx[:, 1]

    # Compute the (first) Franke function
    term_1 = 0.75 * np.exp(-0.25 * ((xx0 - 2) ** 2 + (xx1 - 2) ** 2))
    term_2 = 0.75 * np.exp(
        -1.00 * ((xx0 + 1) ** 2 / 49.0 + (xx1 + 1) ** 2 / 10.0)
    )
    term_3 = 0.50 * np.exp(-0.25 * ((xx0 - 7) ** 2 + (xx1 - 3) ** 2))
    term_4 = 0.20 * np.exp(-1.00 * ((xx0 - 4) ** 2 + (xx1 - 7) ** 2))

    yy = term_1 + term_2 + term_3 - term_4

    return yy

The surface and contour plots of the function are shown below.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 1000)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
yy_2d = fun(xx_2d)

# --- Create two-dimensional plots
fig = plt.figure(figsize=(10, 5))

# Surface
axs_1 = plt.subplot(121, projection='3d')
axs_1.plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    yy_2d.reshape(1000,1000).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs_1.set_xlabel("$x_1$", fontsize=14)
axs_1.set_ylabel("$x_2$", fontsize=14)
axs_1.set_title("Surface plot", fontsize=14)
axs_1.tick_params(axis='both', which='major', labelsize=12)

# Contour
axs_2 = plt.subplot(122)
cf = axs_2.contourf(
    mesh_2d[0], mesh_2d[1], yy_2d.reshape(1000, 1000).T, cmap="plasma"
)
axs_2.set_xlabel("$x_1$", fontsize=14)
axs_2.set_ylabel("$x_2$", fontsize=14)
axs_2.set_title("Contour plot", fontsize=14)
divider = make_axes_locatable(axs_2)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(cf, cax=cax, orientation='vertical')
axs_2.axis('scaled')
axs_2.tick_params(axis='both', which='major', labelsize=12)

fig.tight_layout(pad=6.0)

## Create an interpolating polynomial

The steps to create a multidimensional interpolating polynomial in Minterpy from scratch are similar to those used for the {doc}`one-dimensional case <1d-polynomial-interpolation>`. The steps are:

1. Define the multi-index set of the polynomial
2. Construct an interpolation grid
3. Evaluate the function of interest at the grid points (so-called unisolvent nodes)
4. Create a polynomial in the Lagrange basis
5. Transformation from the Lagrange basis to another basis, preferrably the Newton basis, for further manipulation.

As before, we will go through these steps one at a time, highlighting any differences that arise compared to the one dimensional case.

### Define the multi-index set

As demonstrated in the previous tutorial, multi-index sets in Minterpy are represented by the {py:class}`MultiIndexSet <.core.multi_index.MultiIndexSet>` class.
You can use the class method {py:meth}`from_degree() <.MultiIndexSet.from_degree>` to create a complete multi-index set for a given spatial dimension (first argument), polynomial degree and polynomial degree (second argument).
For multidimensional polynomials, you also need to specify the $l_p$-degree ($> 0.0$) of the multi-index set (third argument).

A complete multi-index set with spatial dimension $m$, polynomial degree $n$, and $l_p$-degree $p$ denoted by $A_{m, n, p}$ includes all multi-indices $\boldsymbol{\alpha}$ that satisfy the condition $\lVert \boldsymbol{\alpha} \rVert_p \leq n$, specifically

$$
A_{m, n, p} = \{ \boldsymbol{\alpha} \in \mathbb{N}^m: \lVert \boldsymbol{\alpha} \lVert_ p = ( \alpha_1^p + \ldots + \alpha_m^p )^{1/p} \leq n \}.
$$

For instance, the complete multi-index set for $m = 2$, $n = 5$, $p = 2.0$:

In [ ]:
m = 2
n = 4
p = 2.0
mi = mp.MultiIndexSet.from_degree(m, n, p)
mi

The set contains the exponents of a multidimensional polynomial. Each column corresponds to the exponents per spatial dimension, while each row indicates the multi-index elements.

To check how many elements the set contains, you can use the built-in `len()` function:

In [ ]:
len(mi)

Several common choices for $l_p$-degree are $1.0$ (_total-degree set_), $2.0$ (_euclidian-degree set_), and $\infty$ (_tensorial set_).
The illustration below shows the difference between these three different choices of $l_p$-degree (for the same spatial dimension and polynomial degree) in relation to the exponents as multi-indices.

In [ ]:
mi_1 = mp.MultiIndexSet.from_degree(m, n, 1.0)
mi_inf = mp.MultiIndexSet.from_degree(m, n, np.inf)

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

axs[0].scatter(mi_1.exponents[:, 0], mi_1.exponents[:, 1], marker='x', color="black")
axs[0].set_xlabel("exponent (dimension = 1)", fontsize=14);
axs[0].set_ylabel("exponent (dimension = 2)", fontsize=14);
axs[0].set_xticks(np.arange(0, n + 1));
axs[0].set_yticks(np.arange(0, n + 1));
axs[0].set_title(rf"m = {m}, n = {n}, p = 1.0", fontsize=16);
axs[0].tick_params(axis='both', which='major', labelsize=12)

axs[1].scatter(mi.exponents[:, 0], mi.exponents[:, 1], marker='x', color="black")
axs[1].set_xlabel("exponent (dimension = 1)", fontsize=14);
axs[1].set_xticks(np.arange(0, n + 1));
axs[1].set_yticks(np.arange(0, n + 1));
axs[1].set_title(rf"m = {m}, n = {n}, p = 2.0", fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=12)

axs[2].scatter(mi_inf.exponents[:, 0], mi_inf.exponents[:, 1], marker='x', color="black")
axs[2].set_xlabel("exponent (dimension = 1)", fontsize=14);
axs[2].set_xticks(np.arange(0, n + 1));
axs[2].set_yticks(np.arange(0, n + 1));
axs[2].set_title(rf"m = {m}, n = {n}, p = $\infty$", fontsize=16);
axs[2].tick_params(axis='both', which='major', labelsize=12)


### Construct an interpolation grid

An interpolating polynomial, one-dimensional or otherwise, lives on an interpolation grid.

To construct an interpolation grid, pass the previously defined multi-index set to the default constructor of the {py:class}`Grid <.cor.grid.Grid>` class:

In [ ]:
grd = mp.Grid(mi)

The unisolvent nodes associated with the interpolating grid has the same spatial dimension as its defining multi-index set. By default, Minterpy generates unisolvent nodes according to the {ref}`Leja-ordered Chebyshev-Lobatto points <fundamentals/interpolation-at-unisolvent-nodes:Generating points>`.
The unisolvent nodes that correspond to the multi-index set are:

In [ ]:
grd.unisolvent_nodes

Each row corresponds to two-dimensional grid points; the first and second column contain all the values of the first and second spatial dimension, respectively.
These are the points at which the function of interest is evaluated; the results are then used construct an interpolating polynomial.

The plots below show three different two-dimensional unisolvent nodes that correspond to three different $l_p$-degrees with the same spatial dimension and polynomial degree.

In [ ]:
mi_1 = mp.MultiIndexSet.from_degree(m, n, 1.0)
grd_1 = mp.Grid(mi_1)
mi_inf = mp.MultiIndexSet.from_degree(m, n, np.inf)
grd_inf = mp.Grid(mi_inf)

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

axs[0].scatter(grd_1.unisolvent_nodes[:, 0], grd_1.unisolvent_nodes[:, 1], marker='x', color="black")
axs[0].set_xlabel("$x_1$", fontsize=14);
axs[0].set_ylabel("$x_2$", fontsize=14);
axs[0].set_xlim([-1.25, 1.25]);
axs[0].set_ylim([-1.25, 1.25]);
axs[0].set_title(rf"m = {m}, n = {n}, p = 1.0", fontsize=16);
axs[0].tick_params(axis='both', which='major', labelsize=12)

axs[1].scatter(grd.unisolvent_nodes[:, 0], grd.unisolvent_nodes[:, 1], marker='x', color="black")
axs[1].set_xlabel("$x_1$", fontsize=14);
axs[1].set_xlim([-1.25, 1.25]);
axs[1].set_ylim([-1.25, 1.25]);
axs[1].set_title(rf"m = {m}, n = {n}, p = 2.0", fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=12)

axs[2].scatter(grd_inf.unisolvent_nodes[:, 0], grd_inf.unisolvent_nodes[:, 1], marker='x', color="black")
axs[2].set_xlabel("$x_1$", fontsize=14);
axs[2].set_xlim([-1.25, 1.25]);
axs[2].set_ylim([-1.25, 1.25]);
axs[2].set_title(rf"m = {m}, n = {n}, p = $\infty$", fontsize=16);
axs[2].tick_params(axis='both', which='major', labelsize=12)


### Evaluate the function at the unisolvent nodes

An interpolating polynomial satisfies the condition that the polynomial values at the unisolvent nodes coincide with the value of the given function at the same nodes.
Calling an instance of {py:class}`Grid <.core.grid.Grid>` with a given function evaluates the function at the unisolvent nodes:

In [ ]:
coeffs = grd(fun)
coeffs

```{note}
Make sure that the given function accepts a two-dimensional array whose each column corresponds to the values per spatial dimension.
```

### Create a polynomial in the Lagrange basis

The function values at the unisolvent nodes are the same as the coefficients $c_{\boldsymbol{\alpha}}$ of a polynomial in the Lagrange basis:

$$
Q(\boldsymbol{x}) = \sum_{\boldsymbol{\alpha} \in A_{m, n, p}} c_{\mathrm{lag}, \boldsymbol{\alpha}} \Psi_{\mathrm{lag}, \boldsymbol{\alpha}}(\boldsymbol{x}),
$$

where $A_{m, n, p}$ is the complete multi-index set and $\Psi_{\mathrm{lag}, \boldsymbol{\alpha}}$'s are the monomials of the Lagrange basis that satisfy:

$$
L_{\boldsymbol{\alpha}}(p_{\boldsymbol{\beta}}) = \delta_{\boldsymbol{\alpha}, \boldsymbol{\beta}}, p_{\boldsymbol{\beta}} \in P_{A_{m, n, p}},
$$

where $\delta{\cdot, \cdots}$ denotes the Kronecker delta.

To create a polynomial in the Lagrange basis from the given grid and coefficients, use the {py:meth}`from_grid() <.polynomials.lagrange_polynomial.LagrangePolynomial.from_grid>` class method of {py:class}`LagrangePolynomial <.polynomials.lagrange_polynomial.LagrangePolynomial>` class:

In [ ]:
lag_poly = mp.LagrangePolynomial.from_grid(grd, coeffs)

### Transform to the Newton basis

Just like the one-dimensional case, to do more with the polynomial in Minterpy, it must first be transformed to the Newton basis such that the polynomial is of the form:

$$
Q(\boldsymbol{x}) = \sum_{\boldsymbol{\alpha} \in A_{m, n, p}} c_{\mathrm{nwt}, \boldsymbol{\alpha}} \, \Psi_{\mathrm{nwt}, \boldsymbol{\alpha}}(\boldsymbol{x}),
$$

where $c_{\mathrm{nwt}, \boldsymbol{\alpha}}$'s are the coefficients of the polynomial in the multidimensional Newton basis and $\Psi_{\mathrm{nwt}, \boldsymbol{\alpha}}$'s are the monomials of the Newton basis. The monomial associated with multi-index element $\boldsymbol{\alpha}$ is defined as follows:

$$
\Psi_{\mathrm{nwt}, \boldsymbol{\alpha}}(\boldsymbol{x}) = \prod_{j = 1}^m \prod_{i = 0}^{\alpha - 1} (x_j - p_{i, j}),
$$

where $p_{i, j}$'s are the interpolation points along dimension $j$ (i.e., the so-called {ref}`generating points / nodes <fundamentals/interpolation-at-unisolvent-nodes:Generating points>` which in multiple dimensions are not the same as the unisolvent nodes).

As before, to transform a polynomial in the Lagrange basis to a polynomial in the Newton basis, use the {py:class}`LagrangeToNewton <.transformations.lagrange.LagrangeToNewton>` class:

In [ ]:
nwt_poly = mp.LagrangeToNewton(lag_poly)()

```{note}
In the above statement, the first part of the call, `mp.LagrangeToNewton(lag_poly)`, creates a transformation instance that converts the given polynomial from the Lagrange basis to the Newton basis.

The second call, which is made without any arguments, performs the transformation and returns the actual Newton polynomial instance.
```

### Evaluate the polynomial

Create a set of random query points in $[-1, 1]^2$:

In [ ]:
xx_test = -1 + 2 * np.random.rand(1000, 2)

Then evaluate the original function at these points

In [ ]:
yy_test = fun(xx_test)
yy_test[:5]

Calling an instance of `NewtonPolynomial` at these query points evaluate the polynomial at the query points:

In [ ]:
yy_poly = nwt_poly(xx_test)
yy_poly[:5]

Note that the input array must have two columns because the polynomial is two dimension.

As expected, evaluating the polynomial on $1'000$ query points returns an array $1'000$ points:

In [ ]:
yy_poly.shape

We can compare the surface plot of the original function and the interpolating polynomial (with the interpolating points):

In [ ]:
# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 250)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
yy_plot_test = fun(xx_2d)
yy_plot_poly = nwt_poly(xx_2d)

# --- Create two-dimensional plots
fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    figsize=(12, 4),
    subplot_kw=dict(projection="3d"),
    layout="compressed",
)

# Original function
axs[0].plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    yy_plot_test.reshape(250, 250).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs[0].set_xlabel("$x_1$", fontsize=14)
axs[0].set_ylabel("$x_2$", fontsize=14)
axs[0].set_title("Original function", fontsize=16)
axs[0].tick_params(axis='both', which='major', labelsize=12)

# Interpolating polynomial
axs[1].plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    yy_plot_poly.reshape(250, 250).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs[1].set_xlabel("$x_1$", fontsize=14)
axs[1].set_ylabel("$x_2$", fontsize=14)
axs[1].set_title("Interpolating polynomial", fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=12)
axs[1].scatter(grd.unisolvent_nodes[:, 0], grd.unisolvent_nodes[:, 1], coeffs, color="k");

### Assess the accuracy of the polynomial

As shown in the plot above, the chosen polynomial degree is not accurate enough to approximate the true function.

The infinity norm provides a measure of the greatest error of the interpolant over the whole domain.
The norm is defined as:

$$
\lVert f - Q \rVert_\infty = \sup_{-1 \leq x \leq 1} \lvert f(x) - Q(x) \rvert
$$

The infinity norm of $Q$ can be approximated using the $1'000$ testing points created above:

In [ ]:
np.max(np.abs(yy_test - yy_poly))

The number indicates that the interpolating polynomial with the choice of $n$ and $p$ is not yet converged.

## Assess the empirical convergence of interpolating polynomials

To assess the convergence of interpolating polynomials, you can once again resort to the high-level function `interpolate()` as a shortcut to the steps taken above.
Let's investigate the convergence of interpolating polynomials with increasing polynomial degrees and three different choices of $l_p$-degrees:



In [ ]:
poly_degrees = np.arange(0, 121, 4)
lp_degrees = [1.0, 2.0, np.inf]
yy_poly = np.empty((len(xx_test), len(poly_degrees), len(lp_degrees)))
for i, n in enumerate(poly_degrees):
    for j, p in enumerate(lp_degrees):
        fx_interp = mp.interpolate(fun, spatial_dimension=2, poly_degree=n, lp_degree=p)
        yy_poly[:, i, j] = fx_interp(xx_test)

In [ ]:
errors = np.max(np.abs(yy_test[:, np.newaxis, np.newaxis] - yy_poly), axis=0)

The convergence of the error is shown below:

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Convergence with respect to the polynomial degrees
axs[0].plot(poly_degrees, errors[:, 0], '-x', label="$p = 1.0$");
axs[0].plot(poly_degrees, errors[:, 1], '-x', label="$p = 2.0$");
axs[0].plot(poly_degrees, errors[:, 2], '-x', label="$p = \infty$");
axs[0].set_yscale("log")
axs[0].legend(fontsize=16);
axs[0].set_xlabel("Polynomial degree ($n$)", fontsize=14)
axs[0].set_ylabel(r"$|| f - Q ||_\infty$", fontsize=14)
axs[0].tick_params(axis='both', which='major', labelsize=12)

# Convergence with respect to the number of coefficients
num_coeffs = np.zeros((len(poly_degrees), len(lp_degrees)))
for i, n in enumerate(poly_degrees):
    for j, p in enumerate(lp_degrees):
        num_coeffs[i, j] = len(mp.MultiIndexSet.from_degree(m, n, p))

axs[1].plot(num_coeffs[:, 0], errors[:, 0], '-x', label="$p = 1.0$");
axs[1].plot(num_coeffs[:, 1], errors[:, 1], '-x', label="$p = 2.0$");
axs[1].plot(num_coeffs[:, 2], errors[:, 2], '-x', label="$p = \infty$");
axs[1].set_yscale("log")
axs[1].set_xscale("log")
axs[1].set_xlabel("Number of coefficients", fontsize=14)
axs[1].tick_params(axis='both', which='major', labelsize=12)

fig.tight_layout()

The surface plots of the original function and the interpolating polynomial for $n = 100$ and $p = 2.0$ (which according to the plots above is numerically converged) are shown below.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# --- Create 2D data
xx_1d = np.linspace(-1.0, 1.0, 250)[:, np.newaxis]
mesh_2d = np.meshgrid(xx_1d, xx_1d)
xx_2d = np.array(mesh_2d).T.reshape(-1, 2)
yy_fun = fun(xx_2d)
yy_poly = mp.interpolate(fun, 2, 128, 1.0)(xx_2d)

# --- Create two-dimensional plots
fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    figsize=(12, 4),
    subplot_kw=dict(projection="3d"),
    layout="compressed",
)

# Original function
axs[0].plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    yy_fun.reshape(250, 250).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs[0].set_xlabel("$x_1$", fontsize=14)
axs[0].set_ylabel("$x_2$", fontsize=14)
axs[0].set_title("Original function", fontsize=16)
axs[0].tick_params(axis='both', which='major', labelsize=12)

# Interpolating polynomial
axs[1].plot_surface(
    mesh_2d[0],
    mesh_2d[1],
    yy_poly.reshape(250, 250).T,
    linewidth=0,
    cmap="plasma",
    antialiased=False,
    alpha=0.5
)
axs[1].set_xlabel("$x_1$", fontsize=14)
axs[1].set_ylabel("$x_2$", fontsize=14)
axs[1].set_title("Interpolating polynomial", fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=12)

## From Interpolant to polynomial

As explained in the {doc}`previous tutorial </getting-started/1d-polynomial-interpolation>`, 
the callable produced by {py:func}`interpolate() <.interpolation.interpolate>`
is not a Minterpy polynomial.

There are convenient methods to represent the interpolant
in one of the Minterpy polynomials.
These methods are summarized in the table below.

|                               Method                                |                    Return the interpolant as a polynomial in the...                     |
|:-------------------------------------------------------------------:|:---------------------------------------------------------------------------------------:|
|    {py:meth}`to_newton() <.interpolation.Interpolant.to_newton>`    |     {py:class}`NewtonPolynomial <.polynomials.newton_polynomial.NewtonPolynomial>`      |
|  {py:meth}`to_lagrange() <.interpolation.Interpolant.to_lagrange>`  |  {py:class}`LagrangePolynomial <.polynomials.lagrange_polynomial.LagrangePolynomial>`   |
| {py:meth}`to_canonical() <.interpolation.Interpolant.to_canonical>` | {py:class}`CanonicalPolynomial <.polynomials.canonical_polynomial.CanonicalPolynomial>` |
| {py:meth}`to_chebyshev() <.interpolation.Interpolant.to_chebyshev>` | {py:class}`CanonicalPolynomial <.polynomials.chebyshev_polynomial.ChebyshevPolynomial>` |

These bases will be the topic of {doc}`/getting-started/polynomial-bases-and-transformations`
in-depth tutorial.

For now to obtain the interpolating polynomial in the Newton basis, call:

In [ ]:
fx_interp.to_newton()

## Summary

In this tutorial, you learned how to create a two-dimensional interpolating polynomial from scratch to approximate a given function then evaluate it at a set of query points in Minterpy.

The steps are:

1. Define a multi-index set
2. Construct an interpolation grid
3. Evaluate the given function on the grid points (i.e., the unisolvent nodes)
4. Construct an interpolating polynomial in the Lagrange basis
5. Transform the polynomial into the equivalent Newton basis

As you observed, these steps mirror those from the {doc}`1d-polynomial-interpolation` tutorial.
A few key points for multidimensional polynomials are:

- you must specify the $l_p$-degree to fully define the multi-index set.
- the function of interest should accept a two-dimensional array, where each row corresponds to a multidimensional point and each column correspondds to the value per dimension.
- when evaluating an interpolating polynomial, the input should also match this two-dimensional array structure.

While the example here is two-dimensional, the same principles apply to polynomials in higher dimensions.

Finally, you saw that for the chosen function, an interpolating polynomial approximate it with sufficiently high polynomial degree. 

---

You've now learned how to construct and work with interpolating polynomials in Minterpy, whether in one or multiple dimensions.

The next two tutorials will explore additional features and capabilities of Minterpy polynomials.